In [ ]:
def LTLUNAB(m, Atf, BAP, clf):
    A0 = []
    A = []
    f = {}
    F = {}
    for item in Atf:
        if m in Atf[item]:
            A0.append(item)
    C = set(A0)
    while(C):
        s = C.pop()
        A.append(s)
        tmp1 = ""
        tmp = ', '.join(list(set(Atf[s]).intersection(BAP)))
        if tmp:
            tmp1 = tmp
            f[s] = {}
            f[s][tmp] = []
        else:
            tmp1 = "0"
            f[s] = {}
            f[s]["0"] = []
        for s1 in Atf:
            if m[0] == "B":
                if m in Atf[s1]:
                    f[s][tmp1].append(s1)
                    if s1 not in A:
                        C.add(s1)
            if m[0] == "D":
                if m in Atf[s1]:
                    f[s][tmp1].append(s1)
                    if s1 not in A:
                        C.add(s1)

    F["F1"] = []
    for a in clf:
        if a[0] == "B":
            for s in A:
                if a in Atf[s] or a[1:] not in Atf[s]:
                    F["F1"].append(s)
        if a[0] == "D":
            for s in A:
                if a not in Atf[s] or a[1:] in Atf[s]:
                    F["F1"].append(s)
    UNAB = {
        "A": A,
        "A0": A0,
        "B(AP)": BAP,
        "f": f,
        "F": F
    }
    return UNAB


m = "D(!p or !q)"
BAP = ["0", "p", "q"]
clf = ["!p", "p", "!q", "q", "(!p or !q)", "!(!p or !q)", "D(!p or !q)", "!D(!p or !q)"]
Atf = {
    "a1": ["p", "!q", "(!p or !q)", "D(!p or !q)"],
    "a2": ["!p", "!q", "(!p or !q)", "D(!p or !q)"],
    "a3": ["!p", "q", "(!p or !q)", "D(!p or !q)"],
    "a4": ["p", "q", "!(!p or !q)", "D(!p or !q)"],
    "a5": ["p", "q", "!(!p or !q)", "!D(!p or !q)"]
}
m2 = "B(!p or !q)"
BAP2 = ["0", "p", "q"]
clf2 = ["!p", "p", "!q", "q", "(!p or !q)", "!(!p or !q)", "B(!p or !q)", "!B(!p or !q)"]
Atf2 = {"a1": ["!p", "!q", "(!p or !q)", "B(!p or !q)"],
        "a2": ["!p", "q", "(!p or !q)", "B(!p or !q)"],
        "a3": ["p", "!q", "(!p or !q)", "B(!p or !q)"],
        "a4": ["p", "q", "!(!p or !q)", "!B(!p or !q)"]}
UNAB = LTLUNAB(m, Atf, BAP, clf)
UNAB2 = LTLUNAB(m2, Atf2, BAP2, clf2)
print(UNAB)
print(UNAB2)

In [ ]:
def merge_automata(UNAB1, UNAB2):
    merged = {}
    merged['A'] = list(set(UNAB1['A'] + UNAB2['A']))
    merged['A0'] = list(set(UNAB1['A0'] + UNAB2['A0']))
    merged['B(AP)'] = UNAB1['B(AP)']  # Assuming B(AP) is the same for both

    # Merge f
    merged['f'] = {}
    for state in merged['A']:
        merged['f'][state] = {}
        if state in UNAB1['f']:
            for key in UNAB1['f'][state]:
                merged['f'][state][key] = UNAB1['f'][state][key]
        if state in UNAB2['f']:
            for key in UNAB2['f'][state]:
                if key in merged['f'][state]:
                    merged['f'][state][key] = list(set(merged['f'][state][key] + UNAB2['f'][state][key]))
                else:
                    merged['f'][state][key] = UNAB2['f'][state][key]

    # Merge F
    merged['F'] = {'F1': list(set(UNAB1['F']['F1'] + UNAB2['F']['F1']))}

    return merged

# Now merge the automata
merged_UNAB = merge_automata(UNAB, UNAB2)
print(merged_UNAB)


In [ ]:
from graphviz import Digraph

def generate_automata_graph(UNAB, filename):
    dot = Digraph(comment='Automaton')

    # Додати стани
    for state in UNAB['A']:
        if state in UNAB['A0']:
            dot.node(state, state, shape='doublecircle')
        else:
            dot.node(state, state)

    # Додати переходи
    for state, transitions in UNAB['f'].items():
        for label, dest_states in transitions.items():
            for dest in dest_states:
                dot.edge(state, dest, label=label)

    dot.render(filename, format='png', cleanup=True)

# Генерація зображень
generate_automata_graph(UNAB, 'UNAB_graph')
generate_automata_graph(UNAB2, 'UNAB2_graph')
generate_automata_graph(merged_UNAB, 'merged_UNAB_graph')
